<a href="https://colab.research.google.com/github/MoMus2000/Airline-Sentiment-Analysis/blob/master/AirlineSentimentpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import collections
import re
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Bidirectional,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('/content/Tweets (1).csv')
# Good practice to shuffle data before use
df = df.reindex(np.random.permutation(df.index))
df = df[['text','airline_sentiment']]


In [3]:
def remove_stopwords(input_text):
  #STOP words are words filtered out before or after processing of NL data
  stopwords_list = stopwords.words('english')
  white_list = ["n't","not","no"]
  words = input_text.split()
  clean_words = [word for word in words if (word not in stopwords_list or word in white_list) and len(word) > 1]
  return "".join(clean_words)

In [4]:
def remove_mentions(input_text):
  return re.sub(r'@\w+','',input_text)

In [5]:
df.text = df.text.apply(remove_stopwords).apply(remove_mentions)
print(df.head())

                                                    text airline_sentiment
1780   .Thenmissednextflightputon.NowI'mgoingLAXinste...          negative
4176   #flighttoday.$700switch.Nonoticenoapology.Wedo...          negative
11670        (RSW)bcblizzardBOS.Toldwayusecredit$200fine          negative
6114   !!leinenkugels#craftbeer#goodflightNorfolk…htt...          positive
10696                                                  !          negative


In [6]:
X_train, X_test,y_train,y_test = train_test_split(df.text,df.airline_sentiment,test_size=0.1,random_state=30)

In [7]:
tokenizer = Tokenizer(num_words = 10000, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
,lower = True, split = " ")
tokenizer.fit_on_texts(X_train)

In [8]:
import collections
print(tokenizer.num_words)
print(collections.Counter(tokenizer.word_counts).most_common(5))
NB_words = 10000

10000
[('t', 1081), ('co', 1078), ('amp', 607), ('http', 500), ('thanks', 214)]


In [9]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
max_sequence_len_xt = max([len(x) for x in X_train_seq])
print(max_sequence_len_xt)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_sequence_len_yt = max([len(x) for x in X_test_seq])
print(max_sequence_len_yt)


14
8


In [10]:
#TRY PADDING PLUS ONE HOT ENCODING
def one_hot_seq(seqs,nb_features=NB_words):
  ohs = np.zeros((len(seqs),nb_features))
  for i,s in enumerate(seqs):
    ohs[i,s]=1
  return ohs
trunc_type='post'
padding_type='post'

In [11]:
# X_train_oh = pad_sequences(X_train_seq, maxlen=max_sequence_len_xt, padding=padding_type, truncating=trunc_type)
# X_test_oh = pad_sequences(X_test_seq, maxlen=max_sequence_len_yt, padding=padding_type, truncating=trunc_type)

In [12]:
X_train_oh = one_hot_seq(X_train_seq)
X_test_oh = one_hot_seq(X_test_seq)
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.fit_transform(y_test)

y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)


In [13]:
#SPLITTING of Validation sets
X_train_rest,X_valid,y_train_rest,y_valid = train_test_split(X_train_oh,y_train_oh,test_size=0.1,random_state=30)

In [14]:
model = Sequential()
model.add(tf.keras.layers.Embedding(10000,100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())

In [15]:
model.fit(X_train_oh,y_train_oh,validation_data=(X_valid,y_valid),epochs=50,batch_size=75)

Epoch 1/50
  9/176 [>.............................] - ETA: 3:20 - loss: 1.0335 - accuracy: 0.5511

KeyboardInterrupt: ignored